In [1]:
from pynq import Overlay
import asyncio
import time
from IPython.display import display, clear_output  

In [2]:
ol = Overlay("./dma_timer_regfile.xsa")
print(ol.ip_dict.keys())
print(ol.is_loaded())

dict_keys(['axi_gpio_0', 'led_blinker_axion_reg_0', 'axi4lite_slave_0', 'axi_intc_0', 'axi_dma_0', 'axi_timer_0', 'zynq_ultra_ps_e_0'])
True


In [3]:
for pin_name, pin_info in ol.interrupt_pins.items():
    print(f"{pin_name}: {pin_info}")

axi_dma_0/mm2s_introut: {'controller': 'axi_intc_0', 'index': 0, 'fullpath': 'axi_dma_0/mm2s_introut'}
xlconcat_intterupts/In0: {'controller': 'axi_intc_0', 'index': 0, 'fullpath': 'xlconcat_intterupts/In0'}
axi_dma_0/s2mm_introut: {'controller': 'axi_intc_0', 'index': 1, 'fullpath': 'axi_dma_0/s2mm_introut'}
xlconcat_intterupts/In1: {'controller': 'axi_intc_0', 'index': 1, 'fullpath': 'xlconcat_intterupts/In1'}
axi_timer_0/interrupt: {'controller': 'axi_intc_0', 'index': 2, 'fullpath': 'axi_timer_0/interrupt'}
xlconcat_intterupts/In2: {'controller': 'axi_intc_0', 'index': 2, 'fullpath': 'xlconcat_intterupts/In2'}


In [4]:
timer0 = ol.axi_timer_0
print(timer0._interrupts)

{'interrupt': {'controller': 'axi_intc_0', 'index': 2, 'fullpath': 'axi_timer_0/interrupt'}}


In [5]:
async def wait_for_timer_interrupt(timer, cycles):  
    """config timer, wait for irq"""  
    # 1. load count number
    timer.register_map.TLR0 = cycles  
      
    # 2. trigger load
    timer.register_map.TCSR0.LOAD0 = 1  
    timer.register_map.TCSR0.LOAD0 = 0  
      
    # 3. enable irq
    timer.register_map.TCSR0.ENIT0 = 1  
      
    # 4. count downward
    timer.register_map.TCSR0.UDT0 = 1  
      
    # 5. enable timer
    timer.register_map.TCSR0.ENT0 = 1  
      
    # wait for irq
    await timer.interrupt.wait()  
      
    # clear irq
    timer.register_map.TCSR0.T0INT = 1

In [6]:
loop = asyncio.get_event_loop()

In [7]:
task = loop.create_task(wait_for_timer_interrupt(timer0, 200000000)) # At 200MHz, 200,000,000 cycles = 1s

In [8]:
!cat /proc/interrupts | grep -E "(fabric|timer)"

 11:    2491316    3916857    2440452    2289718     GICv2  30 Level     arch_timer
 44:         41          0          0          0     GICv2 121 Level     fabric


In [9]:
loop.run_until_complete(task)
print("AXI TIMER IRQ TRIGGERED!")

AXI TIMER IRQ TRIGGERED!


In [10]:
!cat /proc/interrupts | grep -E "(fabric|timer)"

 11:    2491383    3916980    2440521    2289827     GICv2  30 Level     arch_timer
 44:         42          0          0          0     GICv2 121 Level     fabric
